In [3]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)
# 
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

from scripts.data import (
    ml_ratings_df, ml_movies_df, ml_users_df, ml_df, ml_genres,
    bc_ratings_df, bc_books_df, bc_users_df, bc_df,
)

In [2]:
# YOUR AWESOME CODE RIGHT AFTER